Useful links:

* [Continued Fractions](https://mathworld.wolfram.com/topics/ContinuedFractions.html) at Wolfram MathWorld
* Python’s standard [decimal](https://docs.python.org/3/library/decimal.html) package

A _continued fraction_ is a formula that looks like

$$b_0 + {a_1 \over {b_1 + {a_2 \over {b_2 + {a_3 \over {b_3 ...}}}}}}$$

where the $a$s and $b$s are integers. In general, the sequences of terms could be infinite, but to be interesting, the series has to _converge_: as you evaluate more and more terms, you get an answer closer to some number.

Interesting cases are where the $a_n$ and the $b_n$ follow particular patterns. Another special case is the _simple continued fractions_, where the $a_n$ (the _partial numerators_) are all 1.

Just about the simplest possible example: what is the value of

$$x = {1 + {1 \over {1 + {1 \over{1 + {1 \over {1 ...}}}}}}}$$

?

Factoring out the nested recurrence of the whole formula within itself, this equation can be rewritten in a closed form:

$$x = 1 + {1 \over x}$$

from which

$$x - 1 = {1 \over x}$$

or, multiplying both sides by $x$:

$$x^2 - x = 1$$

Which we can solve by “completing the square”:

$$\begin{align}
x^2 - x + {1 \over 4} & = {5 \over 4} \\
& = (x - {1 \over 2})(x - {1 \over 2})
\end{align}$$

So

$${x - {1 \over 2}} = {{± \sqrt 5} \over 2}$$

(remembering that square roots can be positive or negative), from which

$$x = {{± \sqrt 5 + 1} \over 2}$$

which has two answers:

$$\begin{align}
x & = {+ {\sqrt 5 + 1} \over 2} \\
& = 1.618033988...
\end{align}$$

and

$$\begin{align}
x & = {- {\sqrt 5 + 1} \over 2} \\
& = - 0.618033988...
\end{align}$$

These numbers are the well-known “golden number” or “golden ratio”, and its (negated) reciprocal. But clearly the successive approximations to the original continued fraction sum are all positive, so correct answer to the limit value must be the positive one.

In [ ]:
import decimal

Dec = decimal.Decimal
decctx = decimal.getcontext()
decctx.prec = 64

from IPython.display import \
    Latex, \
    display as ipython_display

In the Python code, I will represent the continued fraction as a list of coefficients [$b_0$, $a_1$, $b_1$, $a_2$ ...]. The following function will display the corresponding mathematical formula for a _finite_ list of coefficients:

In [ ]:
def display_continued_fraction(coeffs) :

    def format(coeffs) :
        if len(coeffs) > 2 :
            s = "{%d \over %s}" % (coeffs[1], format(coeffs[2:]))
        elif len(coeffs) == 2 :
            s = "%d" % coeffs[1]
        else :
            s = None
        #end if
        if s != None :
            s = "{%d + %s}" % (coeffs[0], s)
        else :
            s = "%d" % coeffs[0]
        #end if
        return s
    #end format

#begin display_continued_fraction
    ipython_display(Latex("$%s$" % format(coeffs)))
#end format_continued_fraction

while this one will actually evaluate the fraction as a `Decimal` value:

In [ ]:
def evaluate_continued_fraction(coeffs) :
    if len(coeffs) % 2 == 0 :
        coeffs += [1]
    #end if
    result = Dec(coeffs[-1])
    i = len(coeffs) - 1
    while i > 0 :
        result = coeffs[i - 1] / result + coeffs[i - 2]
        i -= 2
    #end while
    return result
#end evaluate_continued_fraction

Example: display the series for $\phi$, to some specified number of terms:

In [ ]:
display_continued_fraction([1] * 20)

Evaluate that series:

In [ ]:
evaluate_continued_fraction([1] * 300)

which is a good approximation to $\phi$:

In [ ]:
(Dec(5).sqrt() + 1) / 2

The following function generates a sequence for $\pi$ derived from the formula for $4 \over \pi$ from [here](https://mathworld.wolfram.com/PiContinuedFraction.html):

In [ ]:
def pi_continued_fraction_1() :
    yield 0
    yield 4
    i = j = 1
    while True :
        yield i
        yield j * j
        i = 2
        j += 2
    #end while
#end pi_continued_fraction_1

I’m too lazy to put a termination condition in the generator, so it will produce a never-ending sequence of terms. Clearly I cannot evaluate such an infinite list, so I use a handy function from the standard [`itertools`](https://docs.python.org/3/library/itertools.html) package to truncate the generator sequence to some specified length:

In [ ]:
import itertools

In [ ]:
print(list(itertools.islice(pi_continued_fraction_1(), 25)))

In [ ]:
display_continued_fraction(list(itertools.islice(pi_continued_fraction_1(), 25)))

In [ ]:
evaluate_continued_fraction(list(itertools.islice(pi_continued_fraction_1(), 100000)))

But this formula seems very slow to converge: even 100000 terms only gives about 5 figures. Compare `math.pi`:

In [ ]:
import math

math.pi

Or compare this function, adapted from the [recipes section](https://docs.python.org/3/library/decimal.html#recipes) of the `decimal` module documentation:

In [ ]:
def decimal_pi():
    with decimal.localcontext() as decctx :
        decctx.prec += 2    # extra digits for intermediate steps
        t = Dec(3)      # substitute 3.0 for regular floats
        lasts, s, n, na, d, da = 0, 3, 1, 0, 0, 24
        nr_steps = 0
        while s != lasts :
            nr_steps += 1
            lasts = s
            n, na = n + na, na + 8
            d, da = d + da, da + 32
            t = t * n / d
            s += t
        #end while
    #end with
    print("nr_steps = %d" % nr_steps)
    return +s               # unary plus applies the new precision
#end decimal_pi

decimal_pi()

As you can see, this converges a lot quicker.

How about using the formula for $\pi \over 2$ instead:

In [ ]:
def pi_continued_fraction_2() :
    yield 2
    yield -2
    i = 1
    while True :
        yield 3
        yield - (i + 1) * (i + 2)
        yield 1
        yield - i * (i + 1)
        i += 2
    #end while
#end pi_continued_fraction_2

In [ ]:
print(list(itertools.islice(pi_continued_fraction_2(), 25)))

In [ ]:
display_continued_fraction(list(itertools.islice(pi_continued_fraction_2(), 25)))

In [ ]:
evaluate_continued_fraction(list(itertools.islice(pi_continued_fraction_2(), 100000)))

Even this has very slow convergence. But the irregular sequence at the start of that page is much better:

In [ ]:
display_continued_fraction([3, 1, 7, 1, 15, 1, 1, 1, 292])

In [ ]:
evaluate_continued_fraction([3, 1, 7, 1, 15, 1, 1, 1, 292])

As you can see, even just a few terms already gives 10 figures! Which seems very good, until you realize that
* there is no simple sequence to the coefficients, so to determine them, you have to know what $\pi$ is to begin with, and
* the coefficients making up that approximation already have more digits than that.

**Conclusion: What is the value of continued fractions?**

Clearly mathematicians have a lot of fun with them. But speaking as someone who does computation on a daily basis, I have to say I don’t think they’re a practical way of evaluating anything. Maybe I’m wrong, and someone who has delved more deeply into them caan offer better examples of how to use them ...